In [1]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

FileNotFoundError: [Errno 2] No such file or directory: 'crypto_market_data.csv'

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
df_market_data_scaled = StandardScaler().fit_transform(df_market_data[['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y']])

In [ ]:
# Create a DataFrame with the scaled data
df_market_scaled=pd.DataFrame(df_market_data_scaled,
                             columns=['price_change_percentage_24h', 'price_change_percentage_7d',
                                       'price_change_percentage_14d', 'price_change_percentage_30d',
                                       'price_change_percentage_60d', 'price_change_percentage_200d',
                                       'price_change_percentage_1y'])

# Copy the crypto names from the original data
df_market_scaled['Crypto_names']=df_market_data.index


# Set the coinid column as index
df_market_scaled=df_market_scaled.set_index('Crypto_names')


# Display sample data
df_market_scaled.head(5)



---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k=list(range(1,11))


In [ ]:
# Create an empty list to store the inertia values
inertia=[]

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, random_state=0)
    k_model.fit(df_market_scaled)
    inertia.append(k_model.inertia_)


In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k_original": k, "inertia": inertia}


# Create a DataFrame with the data to plot the Elbow curve
df_elbow_original = pd.DataFrame(elbow_data)


In [ ]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_plot = df_elbow_original.hvplot.line(
    x="k_original",
    y="inertia",
    title="Elbow Curve",
    xticks=k
)



#### Answer the following question:

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)


In [ ]:
# Fit the K-Means model using the scaled data
model.fit(df_market_scaled)


In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
kmeans_predictions = model.predict(df_market_scaled)


# Print the resulting array of cluster values.
kmeans_predictions

In [ ]:
# Create a copy of the DataFrame
df_market_scaled_prediction=df_market_scaled.copy()


In [ ]:
# Add a new column to the DataFrame with the predicted clusters
df_market_scaled_prediction['Prediction']=kmeans_predictions


# Display sample data
df_market_scaled_prediction.head()


In [ ]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
market_scaled_plot=df_market_scaled_prediction.hvplot.scatter(
                    x="price_change_percentage_24h",
                    y="price_change_percentage_7d",
                    by="Prediction",
                    hover_cols=['Crypto_names']
                )

market_scaled_plot

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)


In [ ]:
# Use the PCA model with `fit_transform` to reduce to
# three principal components.
market_pca = pca.fit_transform(df_market_scaled)

# View the first five rows of the DataFrame.
market_pca[:5]


In [ ]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
pca.explained_variance_ratio_


#### Answer the following question:

**Question:** What is the total explained variance of the three principal components?

**Answer:** 89%

In [ ]:
# Create a new DataFrame with the PCA data.

# Creating a DataFrame with the PCA data
market_pca_df = pd.DataFrame(
    market_pca,
    columns=["PCA1", "PCA2" , "PCA3"]
)
# Copy the crypto names from the original data
market_pca_df["Crypto_names"]=df_market_data.index


# Set the coinid column as index
market_pca_df=market_pca_df.set_index('Crypto_names')


# Display sample data
market_pca_df.head(5)


---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_pca=list(range(1,11))


In [ ]:
# Create an empty list to store the inertia values
inertia=[]


# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k_pca:
    k_model = KMeans(n_clusters=i, random_state=0)
    k_model.fit(market_pca_df)
    inertia.append(k_model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {'k_pca': k_pca , 'inertia': inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_data)
df_elbow_pca.head(7)

In [ ]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_pca_plot = df_elbow_pca.hvplot.line(
    x="k_pca",
    y="inertia",
    title="Elbow Curve",
    xticks=k
)
elbow_pca_plot

#### Answer the following questions:

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:**2


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** The optimal K value for the original data was 4. However, after analyzing the elbow curve plot, the best K value is now determined to be 2.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=2, random_state=0)


In [ ]:
# Fit the K-Means model using the PCA data
model.fit(market_pca_df)


In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
k_predict = model.predict(market_pca_df)

# Print the resulting array of cluster values.
k_predict


In [ ]:
# Create a copy of the DataFrame with the PCA data
market_pca_prediction_df = market_pca_df.copy()


# Add a new column to the DataFrame with the predicted clusters
market_pca_prediction_df['pca_prediction']= k_predict


# Display sample data
market_pca_prediction_df.head()


In [ ]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
market_pca_plot=market_pca_prediction_df.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    by="pca_prediction",
    hover_cols=['Crypto_names']
)
market_pca_plot


### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
elbow_plot + elbow_pca_plot


In [ ]:
# Composite plot to contrast the clusters
market_scaled_plot + market_pca_plot


#### Answer the following question:

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** After visually examining the cluster analysis, it is clear that reducing the number of features for K-Means clustering had a significant impact. Initially, the elbow curve suggested that the best K value for the original data was 4, resulting in 4 clusters. However, two of these clusters contained only one data point each, and the other two clusters were not well separated. By using PPCA and setting K to 2, the resulting clusters were more accurate and clearly defined.